In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
from sklearn.datasets import fetch_california_housing

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
housing = fetch_california_housing()

In [8]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [10]:
class_names = ["T-shit/top", "Trouser", "Pullover", "Dress", "Coat",
              "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [11]:
scaler = StandardScaler()

In [12]:
X_train = scaler.fit_transform(X_train)

In [13]:
X_valid = scaler.transform(X_valid)

In [14]:
X_test = scaler.transform(X_test)

In [15]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
from scipy.stats import reciprocal

In [17]:
from sklearn.model_selection import RandomizedSearchCV

In [18]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [19]:
# 케라스를 사이킷 런으로 돌릴 수 있도록 함
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [20]:
keras_reg.fit(X_train, y_train, epochs=100,
             validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 48us/sample - loss: 1.1399 - val_loss: 24.4309
Epoch 2/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.7393 - val_loss: 3.2896
Epoch 3/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.5374 - val_loss: 0.6080
Epoch 4/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4796 - val_loss: 0.4532
Epoch 5/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4536 - val_loss: 0.4221
Epoch 6/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4373 - val_loss: 0.4144
Epoch 7/100
11610/11610 [==============================] - ETA: 0s - loss: 0.429 - 0s 26us/sample - loss: 0.4256 - val_loss: 0.4025
Epoch 8/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4178 - val_loss: 0.3938
Epoch 9/100
11610/11610 [==============================] - 0s 

In [21]:
param_distribs = {
    "n_hidden": [1, 2, 3], 
    "n_neurons": np.arange(40, 45),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

In [22]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)

In [23]:
# rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
#                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [24]:
%%time
rnd_search_cv.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=2)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.001683454924600351, n_hidden=1, n_neurons=42 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


7740/7740 [==============================] - 0s 62us/sample - loss: 1.7605 - val_loss: 0.8180
Epoch 2/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.7492 - val_loss: 0.6902
Epoch 3/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.6682 - val_loss: 0.6794
Epoch 4/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.6227 - val_loss: 0.6877
Epoch 5/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5881 - val_loss: 0.5605
Epoch 6/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5563 - val_loss: 0.5246
Epoch 7/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.5309 - val_loss: 0.5171
Epoch 8/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5110 - val_loss: 0.5072
Epoch 9/10
7740/7740 [==============================] - 0s 26us/sample - loss: 0.4933 - val_loss: 0.4714
Epoch 10/10
3870/3870 [==============================] - 0s 12us/s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


7740/7740 [==============================] - 0s 58us/sample - loss: 1.5698 - val_loss: 0.8937
Epoch 2/10
7740/7740 [==============================] - 0s 29us/sample - loss: 0.7232 - val_loss: 0.6627
Epoch 3/10
7740/7740 [==============================] - 0s 29us/sample - loss: 0.6623 - val_loss: 0.7438
Epoch 4/10
3870/3870 [==============================] - 0s 12us/sample - loss: 0.6249
[CV]  learning_rate=0.001683454924600351, n_hidden=1, n_neurons=42, total=   1.2s
[CV] learning_rate=0.001683454924600351, n_hidden=1, n_neurons=42 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/10
7740/7740 [==============================] - 0s 58us/sample - loss: 2.0150 - val_loss: 7.4186
Epoch 2/10
7740/7740 [==============================] - 0s 29us/sample - loss: 0.7936 - val_loss: 3.2821
Epoch 3/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6761 - val_loss: 1.3613
Epoch 4/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6329 - val_lo

...

7740/7740 [==============================] - 0s 29us/sample - loss: 0.8471 - val_loss: 0.7278
Epoch 4/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.7128 - val_loss: 0.6497
Epoch 5/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.6644 - val_loss: 0.6537
Epoch 6/10
7740/7740 [==============================] - 0s 29us/sample - loss: 0.6384 - val_loss: 0.6023
Epoch 7/10
7740/7740 [==============================] - 0s 29us/sample - loss: 0.6171 - val_loss: 0.5971
Epoch 8/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5997 - val_loss: 0.5932
Epoch 9/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5844 - val_loss: 0.5976
Epoch 10/10
3870/3870 [==============================] - 0s 13us/sample - loss: 0.5649
[CV]  learning_rate=0.0006930605663535878, n_hidden=1, n_neurons=40, total=   2.7s
[CV] learning_rate=0.0012178834831452913, n_hidden=2, n_neurons=44 ...
Train on 7740 samples, validate on 

7740/7740 [==============================] - 0s 30us/sample - loss: 0.6325 - val_loss: 7.9812
Epoch 3/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.5663 - val_loss: 1.0344
Epoch 4/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.4517 - val_loss: 0.5222
Epoch 5/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.4178 - val_loss: 0.4184
Epoch 6/10
7740/7740 [==============================] - 0s 31us/sample - loss: 0.4027 - val_loss: 0.3701
Epoch 7/10
7740/7740 [==============================] - 0s 31us/sample - loss: 0.3921 - val_loss: 0.3628
Epoch 8/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.3841 - val_loss: 0.3652
Epoch 9/10
3870/3870 [==============================] - 0s 14us/sample - loss: 0.3897
[CV]  learning_rate=0.005021425736625637, n_hidden=2, n_neurons=43, total=   2.5s
[CV] learning_rate=0.005021425736625637, n_hidden=2, n_neurons=43 ....
Train on 7740 samples, validate on 38

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.0min finished


11610/11610 [==============================] - 1s 60us/sample - loss: 0.6185 - val_loss: 1.9088
Epoch 2/10
11610/11610 [==============================] - 0s 28us/sample - loss: 0.4102 - val_loss: 0.4612
Epoch 3/10
11610/11610 [==============================] - 0s 29us/sample - loss: 0.3702 - val_loss: 0.3281
Epoch 4/10
11610/11610 [==============================] - 0s 29us/sample - loss: 0.3536 - val_loss: 0.3676
Epoch 5/10
11610/11610 [==============================] - 0s 28us/sample - loss: 0.3467 - val_loss: 0.5500
Wall time: 1min 4s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000022953F74780>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000022954009278>,
                                        'n_hidden': [1, 2, 3],
                                        'n_neurons': array([40, 41, 42, 43, 44])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [25]:
rnd_search_cv.best_estimator_

In [26]:
rnd_search_cv.score(X_test, y_test)

5160/5160 [==============================] - 0s 16us/sample - loss: 0.3432


-0.34319735215615854

In [27]:
model = rnd_search_cv.best_estimator_.model

In [28]:
model.evaluate(X_test, y_test)

5160/5160 [==============================] - 0s 13us/sample - loss: 0.3432


0.34319735215615854

In [29]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [30]:
import tensorflow as tf

In [31]:
from tensorflow import keras

In [32]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [33]:
# 초기화

In [34]:
[ name for name in dir(keras.initializers) if not name.startswith("_") ]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'Identity',
 'Initializer',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'serialize',
 'zeros']

In [35]:
keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

In [36]:
[ m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'tanh']

In [37]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [38]:
X_train_full = X_train_full / 255.0

In [39]:
X_test = X_test / 255.0

In [40]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]

In [41]:
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [42]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation='softmax')
])

In [43]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=keras.optimizers.SGD(lr=1e-3),
             metrics=['accuracy'])

In [44]:
hist = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 3s 53us/sample - loss: 1.2810 - accuracy: 0.6205 - val_loss: 0.8869 - val_accuracy: 0.7160
Epoch 2/30
55000/55000 [==============================] - 3s 47us/sample - loss: 0.7952 - accuracy: 0.7369 - val_loss: 0.7132 - val_accuracy: 0.7626
Epoch 3/30
55000/55000 [==============================] - 3s 47us/sample - loss: 0.6817 - accuracy: 0.7725 - val_loss: 0.6385 - val_accuracy: 0.7896
Epoch 4/30
55000/55000 [==============================] - 3s 46us/sample - loss: 0.6219 - accuracy: 0.7941 - val_loss: 0.5931 - val_accuracy: 0.8016
Epoch 5/30
55000/55000 [==============================] - 2s 45us/sample - loss: 0.5830 - accuracy: 0.8073 - val_loss: 0.5607 - val_accuracy: 0.8170
Epoch 6/30
55000/55000 [==============================] - 2s 45us/sample - loss: 0.5552 - accuracy: 0.8173 - val_loss: 0.5355 - val_accuracy: 0.8238
Epoch 7/30
55000/55000 [==============================] -

In [45]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation='softmax')
])

In [46]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer=keras.optimizers.SGD(lr=1e-2),
             metrics=['accuracy'])

In [47]:
hist = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 4s 65us/sample - loss: 0.6982 - accuracy: 0.7684 - val_loss: 0.5053 - val_accuracy: 0.8306
Epoch 2/100
55000/55000 [==============================] - 3s 55us/sample - loss: 0.4836 - accuracy: 0.8319 - val_loss: 0.4640 - val_accuracy: 0.8408
Epoch 3/100
55000/55000 [==============================] - 3s 55us/sample - loss: 0.4410 - accuracy: 0.8453 - val_loss: 0.4434 - val_accuracy: 0.8514
Epoch 4/100
55000/55000 [==============================] - 3s 55us/sample - loss: 0.4126 - accuracy: 0.8551 - val_loss: 0.4152 - val_accuracy: 0.8578
Epoch 5/100
55000/55000 [==============================] - 3s 55us/sample - loss: 0.3931 - accuracy: 0.8618 - val_loss: 0.3821 - val_accuracy: 0.8672
Epoch 6/100
55000/55000 [==============================] - 3s 58us/sample - loss: 0.3782 - accuracy: 0.8665 - val_loss: 0.3781 - val_accuracy: 0.8704
Epoch 7/100
55000/55000 [==========================

Epoch 55/100
55000/55000 [==============================] - 9s 156us/sample - loss: 0.1388 - accuracy: 0.9514 - val_loss: 0.3231 - val_accuracy: 0.8948
Epoch 56/100
55000/55000 [==============================] - 6s 103us/sample - loss: 0.1368 - accuracy: 0.9518 - val_loss: 0.3010 - val_accuracy: 0.8988
Epoch 57/100
55000/55000 [==============================] - 4s 73us/sample - loss: 0.1334 - accuracy: 0.9531 - val_loss: 0.3009 - val_accuracy: 0.9002
Epoch 58/100
55000/55000 [==============================] - 3s 60us/sample - loss: 0.1297 - accuracy: 0.9546 - val_loss: 0.3020 - val_accuracy: 0.9018
Epoch 59/100
55000/55000 [==============================] - 3s 59us/sample - loss: 0.1282 - accuracy: 0.9551 - val_loss: 0.3132 - val_accuracy: 0.8996
Epoch 60/100
55000/55000 [==============================] - 4s 73us/sample - loss: 0.1253 - accuracy: 0.9568 - val_loss: 0.3146 - val_accuracy: 0.8960
Epoch 61/100
55000/55000 [==============================] - 5s 85us/sample - loss: 0.1239 - 

In [48]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation='selu', kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation='selu',
                                 kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [49]:
[ name for name in dir(keras.layers) if not name.startswith("_")]

['AbstractRNNCell',
 'Activation',
 'ActivityRegularization',
 'Add',
 'AdditiveAttention',
 'AlphaDropout',
 'Attention',
 'Average',
 'AveragePooling1D',
 'AveragePooling2D',
 'AveragePooling3D',
 'AvgPool1D',
 'AvgPool2D',
 'AvgPool3D',
 'BatchNormalization',
 'Bidirectional',
 'Concatenate',
 'Conv1D',
 'Conv2D',
 'Conv2DTranspose',
 'Conv3D',
 'Conv3DTranspose',
 'ConvLSTM2D',
 'Convolution1D',
 'Convolution2D',
 'Convolution2DTranspose',
 'Convolution3D',
 'Convolution3DTranspose',
 'Cropping1D',
 'Cropping2D',
 'Cropping3D',
 'Dense',
 'DenseFeatures',
 'DepthwiseConv2D',
 'Dot',
 'Dropout',
 'ELU',
 'Embedding',
 'Flatten',
 'GRU',
 'GRUCell',
 'GaussianDropout',
 'GaussianNoise',
 'GlobalAveragePooling1D',
 'GlobalAveragePooling2D',
 'GlobalAveragePooling3D',
 'GlobalAvgPool1D',
 'GlobalAvgPool2D',
 'GlobalAvgPool3D',
 'GlobalMaxPool1D',
 'GlobalMaxPool2D',
 'GlobalMaxPool3D',
 'GlobalMaxPooling1D',
 'GlobalMaxPooling2D',
 'GlobalMaxPooling3D',
 'Input',
 'InputLayer',
 'Input

In [50]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=['accuracy'])

In [51]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)

X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [53]:
hist = model.fit(X_train_scaled, y_train, epochs=5, validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 24s 443us/sample - loss: 1.3436 - accuracy: 0.4844 - val_loss: 0.8793 - val_accuracy: 0.6660
Epoch 2/5
55000/55000 [==============================] - 21s 384us/sample - loss: 0.9516 - accuracy: 0.6285 - val_loss: 0.7473 - val_accuracy: 0.7112
Epoch 3/5
55000/55000 [==============================] - 21s 383us/sample - loss: 0.7206 - accuracy: 0.7265 - val_loss: 0.8219 - val_accuracy: 0.6916
Epoch 4/5
55000/55000 [==============================] - 21s 385us/sample - loss: 0.6319 - accuracy: 0.7713 - val_loss: 0.6082 - val_accuracy: 0.7890
Epoch 5/5
55000/55000 [==============================] - 22s 393us/sample - loss: 0.6742 - accuracy: 0.7607 - val_loss: 0.6845 - val_accuracy: 0.7598


In [54]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation='relu', kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation='relu',
                                 kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [55]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=['accuracy'])

In [56]:
hist = model.fit(X_train_scaled, y_train, epochs=5, validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 23s 419us/sample - loss: 2.3026 - accuracy: 0.0991 - val_loss: 2.3026 - val_accuracy: 0.0914
Epoch 2/5
55000/55000 [==============================] - 20s 367us/sample - loss: 2.3026 - accuracy: 0.0989 - val_loss: 2.3026 - val_accuracy: 0.0914
Epoch 3/5
55000/55000 [==============================] - 20s 363us/sample - loss: 2.3026 - accuracy: 0.1000 - val_loss: 2.3027 - val_accuracy: 0.0914
Epoch 4/5
55000/55000 [==============================] - 20s 364us/sample - loss: 2.3026 - accuracy: 0.1008 - val_loss: 2.3027 - val_accuracy: 0.0914
Epoch 5/5
55000/55000 [==============================] - 20s 364us/sample - loss: 2.3026 - accuracy: 0.1008 - val_loss: 2.3027 - val_accuracy: 0.0914
